In [1]:
import numpy as np
import pandas as pd

In [2]:
food = '/Users/remi/Documents/FoodFact/foodfact.csv'

In [14]:
df_food = pd.read_csv(food, sep="\t")

/var/folders/bt/y6hlhl2x74q8ty65t1qmh7sr0000gq/T/ipykernel_1611/4268972478.py:1: DtypeWarning: Columns (0,3,5,19,20,24,25,26,27,28,35,36,37,38,39,48) have mixed types. Specify dtype option on import or set low_memory=False.
  df_food = pd.read_csv(food, sep="\t")


In [21]:
columns_indices = [0, 3, 5, 19, 20, 24, 25, 26, 27, 28, 35, 36, 37, 38, 39, 48]
selected_columns = df_food.iloc[:, columns_indices]

In [25]:
selected_columns.isna().mean().sort_values()

last_modified_t                   0.000000
created_t                         0.000009
code                              0.000072
manufacturing_places              0.886209
manufacturing_places_tags         0.886228
emb_codes                         0.908639
emb_codes_tags                    0.908649
allergens                         0.911638
traces                            0.924080
traces_fr                         0.924083
traces_tags                       0.924155
cities_tags                       0.936653
first_packaging_code_geo          0.941382
ingredients_from_palm_oil_tags    0.984927
cities                            0.999928
allergens_fr                      0.999941
dtype: float64

In [5]:
test = []
for col in df_food.columns:
    test.append(col)


In [6]:
column_select = pd.DataFrame((df_food.isna().mean()*100).sort_values(ascending=False))

In [7]:
column_select.reset_index(inplace=True)

In [8]:
column_select.rename(columns={0: "zero values percent"}, inplace=True)

In [9]:
column_select.dtypes

index                   object
zero values percent    float64
dtype: object

In [10]:
columns_zero_values = []

for col in column_select["zero values percent"]:
    if col > 90:
        columns_zero_values.append(column_select["index"])

list(columns_zero_values)

[0                        water-hardness_100g
 1                              no_nutriments
 2      ingredients_that_may_be_from_palm_oil
 3                         nutrition_grade_uk
 4                         nervonic-acid_100g
                        ...                  
 157                         created_datetime
 158                                created_t
 159                                  creator
 160                   last_modified_datetime
 161                          last_modified_t
 Name: index, Length: 162, dtype: object,
 0                        water-hardness_100g
 1                              no_nutriments
 2      ingredients_that_may_be_from_palm_oil
 3                         nutrition_grade_uk
 4                         nervonic-acid_100g
                        ...                  
 157                         created_datetime
 158                                created_t
 159                                  creator
 160                   last_modified_d

In [11]:
columns_zero_values

[0                        water-hardness_100g
 1                              no_nutriments
 2      ingredients_that_may_be_from_palm_oil
 3                         nutrition_grade_uk
 4                         nervonic-acid_100g
                        ...                  
 157                         created_datetime
 158                                created_t
 159                                  creator
 160                   last_modified_datetime
 161                          last_modified_t
 Name: index, Length: 162, dtype: object,
 0                        water-hardness_100g
 1                              no_nutriments
 2      ingredients_that_may_be_from_palm_oil
 3                         nutrition_grade_uk
 4                         nervonic-acid_100g
                        ...                  
 157                         created_datetime
 158                                created_t
 159                                  creator
 160                   last_modified_d